In [1]:
from OptimalControl import misc
from qutip_enhanced import *
import os
import shutil
import OptimalControl
import matlab.engine

import StringIO

class DynPython(object):

    def __init__(self, dynamo_path, add_slices=None, sample_frequency=12e3):
        self.add_slices = add_slices
        self.sample_frequency = sample_frequency
        self.dynamo_path = dynamo_path
        self.eng = matlab.engine.start_matlab()
        self.out = StringIO.StringIO()
        self.err = StringIO.StringIO()
        self.already_printed = ''

    @property
    def fields_names_list(self):
        if hasattr(self, '_fields_names'):
            return self._fields_names
        else:
            return ["{}".format(i) for i in range(len(self.export_mask_list))]

    @fields_names_list.setter
    def fields_names_list(self, val):
        self._fields_names = val

    def print_out(self):
        to_print = self.out.getvalue()
        print(to_print.strip(self.already_printed))
        self.already_printed = to_print

    @property
    def dyn(self):
        return self.eng.workspace["dyn"]

    def set_dyn(self, task, initial_gate, CROT, H_drift_list, H_ctrl_list, weight):
        self.eng.addpath(self.eng.genpath(self.dynamo_path))
        self._dyn = self.eng.dynamo(task, initial_gate, CROT, H_drift_list, H_ctrl_list, weight, stdout=self.out, stderr=self.err)
        self.eng.workspace["dyn"] = self._dyn
        self.print_out()

    @property
    def export_mask_list(self):
        if not hasattr(self, '_export_mask_list') or self._export_mask_list is None:
            return [self.get_mask()[:self.n_bins, :]]
        else:
            return self._export_mask_list

    @export_mask_list.setter
    def export_mask_list(self, val):
        columns = self.get_mask().shape[1] - 1 # last column of the optimize mask are timeslices
        if val is None:
            pass
        if type(val) is list:
            for i, v in enumerate(val):
                if v.shape != (self.n_bins, columns):
                    raise Exception('Error: i: {}, v.shape: {}, bins: {}, columns: {}'.format(i, v.shape, self.n_bins, columns))
                if sum(~np.all(v == 0, axis=0)) != 2:
                    raise Exception('Each export_mask must one x and one y row totaling in two nonzero columns. Here {} colums are nonzero'.format(len(~np.all(v == 0, axis=0))))
        else:
            raise Exception('Error, {}'.format(val))
        self._export_mask_list = val

    def export_mask_columns(self, n):
        return np.where(~np.all(self.export_mask_list[n] == 0, axis=0))[0]

    def export_mask_rows(self, n):
        return np.where(self.export_mask_list[n][:, np.where(~np.all(self.export_mask_list[n] == 0, axis=0))[0][0]])[0]

    def set_labels(self, title, dims, c_labels):
        self.eng.eval("dyn.system.set_labels('{}', {}, {{{}}})".format(title, dims, ", ".join("'{}'".format(i) for i in c_labels)), stdout=self.out, stderr=self.err, nargout=0)

    def get_mask(self, optimize_times=None):
        if optimize_times is None:
            return np.array(self.eng.eval("dyn.opt.control_mask"))
        else:
            return np.array(self.eng.full_mask(self.dyn, int(optimize_times)))

    def seq_init(self, n_bins, add_slices, TL, control_type, control_par):
        self.eng.seq_init(self.dyn, float(n_bins + add_slices), TL, control_type, control_par, stdout=self.out, stderr=self.err, nargout=0)
        self.print_out()

    def set_controls(self, fields):
        self.eng.set_controls(self.dyn, fields, stdout=self.out, stderr=self.err, nargout=0)
        self.print_out()

    def open_ui(self):
        self.eng.ui_open(self.dyn, nargout=0)

    def search(self, mask, options=None):
        options = {} if options is None else options
        mask = matlab.logical(mask.tolist())
        self.eng.search(self.dyn, mask, options, stdout=self.out, stderr=self.err, nargout=0)
        self.print_out()

    @property
    def n_bins(self):
        return len(self.get_mask()) - self.add_slices

    @property
    def times_fields_mhz(self):
        out = np.array(self.eng.eval("dyn.export"))
        out[:, 1:] *= 2 * np.pi
        return out

    def times(self, n):
        """
        :param sample_frequency: in MHz
        """
        return self.times_fields_mhz[self.export_mask_rows(n), 0]

    @property
    def times_bins(self):
        return self.times_fields_mhz[:self.n_bins, 0]

    @property
    def times_add_slices(self):
        return self.times_fields_mhz[self.n_bins:, 0]

    def fields_xy_mhz(self, n):
        return self.times_fields_mhz[self.export_mask_rows(n), 1:][:, ~np.all(self.export_mask_list[n] == 0, axis=0)]

    def fields_aphi_mhz(self, n):
        return misc.xy2aphi(self.fields_xy_mhz(n))

    @property
    def fields_add_slices_mhz(self):
        out = self.times_fields_mhz[self.n_bins:, -self.add_slices:]
        if self.add_slices > 1:
            out = np.diag(out)
        return out

    @property
    def add_slices_angles(self):
        if self.add_slices > 0:
            return (2 * np.pi * self.times_add_slices*self.fields_add_slices_mhz) % (2 * np.pi)

    def save_times_fields_mhz(self, path):
        np.savetxt(path, np.around(self.times_fields_mhz, 8), fmt='%+1.4e')

    def save_times_fields_xy(self, n=None, path=None):
        t = self.round2float(self.times(n), 1/self.sample_frequency)
        txy = np.column_stack([t, self.fields_xy_mhz(n)])
        np.savetxt(path, txy, fmt='%+1.4e')

    def save_times_fields_aphi(self, n=None, path=None):
        t = self.round2float(self.times(n), 1/self.sample_frequency)
        txy = np.column_stack([t, self.fields_aphi_mhz(n)])
        np.savetxt(path, txy, fmt='%+1.4e')

    def save_add_slices_angles(self, path):
        asa = self.add_slices_angles
        if asa is not None:
            np.savetxt(path, asa, fmt='%+1.4e')
        print "add_slices_angles saved."

    def round2float(self, arr, val):
        return np.around(arr/val) * val

    def save_dynamo_fields(self, base_path):
        self.save_times_fields_mhz("{}\\fields.dat".format(base_path))
        for n, name in enumerate(self.fields_names_list):
            self.save_times_fields_aphi(n=n, path="{}\\{}.dat".format(base_path, name))
            print "Fields {} saved.".format(name)
        self.save_add_slices_angles(path="{}\\add_slices_angles.dat".format(base_path))

    def save(self, script_path=None, script_code=None, substr=None):
        script_path = script_path.split('.')[0] + '.py'
        dn = os.path.dirname(script_path)
        dns = dn + '\\' + misc.nowstr() + substr
        if not os.path.isdir(dns):
            os.mkdir(dns)
        with open(dns + '\\' + os.path.basename(script_path), "w") as text_file:
            text_file.write(script_code)
        shutil.copytree(os.path.dirname(OptimalControl.__file__), dns+'\\OptimalControl')
        print "Optimal control saved."
        import qutip_enhanced
        shutil.copytree(os.path.dirname(qutip_enhanced.__file__), dns + '\\qutip_enhanced')
        print "qutip_enhanced saved."
        self.save_dynamo_fields(base_path=dns)

def rotop(*args, **kwargs):
    return 2*np.pi*get_rot_matrix_all_spins(*args, **kwargs).data.todense()

def load(eng, directory):
    """
    :param directory: directory in which mat file and script and so on lies
    :return:
    """
    eng.addpath(eng.genpath(directory))

QuTiP warning: Cython setup failed: No module named Cython


In [2]:
import numpy as np

def mw_locations(n_pi=None, spt=None):
    """
    outputs pulse_locations - np.one, e.g. n_pi = 20, spt = 1 --> [1, 4, 7, 10.. 58]
    """
    return np.arange(spt, 2*n_pi*spt + n_pi, 2*spt + 1, dtype=int)

def rf_locations(n_pi=None, spt=None):
    return np.where(l_active(n_pi=n_pi, spt=spt) == 1)[0]

def l_active(n_pi=None, spt=None):
    out = np.ones([2 * n_pi * spt + n_pi], dtype=bool)
    out[mw_locations(n_pi=n_pi, spt=spt)] = 0.
    return out

def tau(T=None, n_pi=None, spt=None):
    return T/(2*n_pi*spt)

def alpha(T=None, n_pi=None, n=None, target_Azz=None, spt=None, rabi_period=None):
    delta_alpha = 2*np.pi*target_Azz*(tau(T=T, n_pi=n_pi, spt=spt)*spt + 0.25*rabi_period)
    return int(np.ceil(n/2.))*(delta_alpha + np.pi) % (2*np.pi)

def u_array(T=None, n_pi=None, spt=None, target_Azz=None, rabi_period=None):
    omega = 0.5 / T
    u_phi = [alpha(T=T, n_pi=n_pi, n=n, target_Azz=target_Azz, spt=spt, rabi_period=rabi_period) for n in range(2*n_pi)]
    u_x = np.array([omega * np.cos(phi) for phi in u_phi])
    u_x = np.tile([u_x for i in range(spt)], 1).T.reshape(-1)
    u_y = np.array([omega * np.sin(phi) for phi in u_phi])
    u_y = np.tile([u_y for i in range(spt)], 1).T.reshape(-1)
    return np.array([u_x, u_y]).T

def u_array_pi(**kwargs):
    return np.insert(u_array(**kwargs), mw_locations(n_pi=kwargs['n_pi'], spt=kwargs['spt']) - np.arange(kwargs['n_pi']), np.array([0, 0]), 0)

def delete_pi(u_array=None, n_pi=None, spt=None):
    return np.delete(u_array, mw_locations(n_pi=n_pi, spt=spt), axis=0)

def mask_rf(n_pi=None, spt=None, add_slices=None):
    rl = rf_locations(n_pi=n_pi, spt=spt)
    out = np.zeros((len(l_active(n_pi=n_pi, spt=spt)), 4 + add_slices), dtype=bool)
    out[rl, 2] = True
    out[rl, 3] = True
    return out

def mask_mw(n_pi=None, spt=None, add_slices=None):
    rl = mw_locations(n_pi=n_pi, spt=spt)
    out = np.zeros((len(l_active(n_pi=n_pi, spt=spt)), 4 + add_slices), dtype=bool)
    out[rl, 0] = True
    out[rl, 1] = True
    return out

def mask(**kwargs):
    return mask_rf(**kwargs) + mask_mw(**kwargs)

In [3]:
import matlab.engine
import sys
sys.path.append(r"D:\Dropbox\Dokumente\PI3\Code\Python\GRAPE_Philipp\Basti\cnot_kdd\cnot_kdd")
#import dd_alpha coming from above
import numpy as np
import OptimalControl as OC
import matplotlib.pyplot as plt
import matplotlib
from qutip_enhanced import *

try:
    with open(__file__, 'r') as f:
        script_code = f.read()
except:
    script_code = "not started from file"
    import os
    __file__ = os.getcwd() + '\\no_file.py'

rabi_period = 0.2
dd_phases = OC.misc.phases_kdd
n_pi = len(dd_phases)
spt = 1
n_bins_tau = 2*spt*n_pi
n_bins = n_pi + n_bins_tau
T = 400.
T_lb = .0001
dims = [2, 2, 2, 2, 2] # e, n14, 13c414, 13c90, 13c13

n_type = None if len(dims) < 5 else '14n'

target_spin = '13C90'
if target_spin == '13C13':
    target_spin_number = 4
    target_Azz = 0.0123
elif target_spin == '13C90':
    target_spin_number = 3
    if len(dims) == 2:
        target_spin_number += 1
    target_Azz = 0.0888

target_spin_number -= 5 - len(dims)

######################################################################################
# control fields
######################################################################################
maxrabis = {'13C414 mS0': 0.0241,
            '13C90 mS0': 0.0241,
            '13C13 mS0': 0.0241,
            '13C414 mS-1': 0.0253,
            '13C90 mS-1': 0.0249,
            '13C13 mS-1': 0.0241,
            '14N+1 mS0': 0.0139,
            '14N+1 mS-1': 0.0051}
rabi_ratios = dict([(key, maxrabis[key]/maxrabis['{} mS-1'.format(target_spin)]) for key in maxrabis.keys()])

def rr(spin_number, ms):
    if len(dims) == 5:
        d = {1: '14N+1', 2: '13C414', 3: '13C90', 4: '13C13'}
    elif len(dims) == 4:
        d = {1: '13C414', 2: '13C90', 3: '13C13'}
    elif len(dims) == 3:
        d = {1: '13C90', 2: '13C13'}
    elif len(dims) == 2:
        d = {1: target_spin}
    else:
        raise Exception('Error')
    return rabi_ratios["{} mS{}".format(d[spin_number], ms)]

def fBc(spin_number, ms, axis):
    esn = 0 if ms=='0' else 1
    return rr(spin_number, ms) * OC.dynamo_helpers.rotop(dims, rotated_spin=spin_number, selective_to={0:[esn]}, rotation_axis={axis: 1}, angle=np.pi)

L_Bc_xy_e = [OC.dynamo_helpers.rotop(dims, rotated_spin=0, rotation_axis={axis: 1}, angle=np.pi) for axis in ['x', 'y']]
L_Bc_xy_nuc = [qsum([qsum([fBc(spin_number, ms, axis) for ms in ['0', '-1']]) for spin_number in range(1, len(dims))]) for axis in ['x', 'y']]

zd = dict(rotation_axis={'z': 1}, angle=np.pi)
L_Bc_z = [OC.dynamo_helpers.rotop(dims, rotated_spin=i, **zd) for i in range(1, len(dims))]

nz = len(L_Bc_z)
L_Bc = L_Bc_xy_e + L_Bc_xy_nuc + L_Bc_z
n_ctrl = len(L_Bc)
add_slices = n_ctrl - 4

maxrabis = np.array([ 1/rabi_period, 1/rabi_period, 0.01/np.sqrt(2), 0.01/np.sqrt(2)] + [1000000.]*nz).tolist()
fields = np.repeat([[0.0, 0.0, 0.000, 0.000] + (0.001*np.ones(nz)).tolist()], n_bins+add_slices, axis=0)
for i, phi in enumerate(OC.misc.phases_kdd):
    fields[spt*(1 + 2*i) + i, :2] = 1 / rabi_period*np.array([np.cos(phi), np.sin(phi)])

D = np.prod(dims)

control_type = 'm'*n_ctrl
control_par = [matlab.double([-i, 2*i]) for i in maxrabis]

p_scales_base = [np.array([i, i] + [1.] * (n_ctrl-2)) for i in [1.]]#[0.995, 1., 1.005]]
dw = 0.00002
detunings_base = [0.0]#np.linspace(-dw/2., dw/2., 3)
detunings = np.repeat(detunings_base, len(p_scales_base))
p_scales = np.tile(p_scales_base, (len(detunings_base), 1))
weight = OC.misc.multi_gaussian(detunings, sigma=0.3*dw, L_mu=np.array([0.0]))
weight = weight/ np.sum(weight)
weight = matlab.double(weight.tolist())

nvham = nv_hamilton.NVHam(magnet_field={'z': .6637734172532993},
                          n_type=n_type,
                          nitrogen_levels=[0, 1],
                          electron_levels=[1, 2],
                          hf_para_n={'14n': -2.16564488209},
                          gamma={'13c': 10.703684516, '14n': 3.07566840499, 'e': -28024.9536})
if len(dims) > 3:
    nvham.add_spin(np.diag([0,0,0.413]), nvham.h_13c(), [0, 1])
if len(dims) > 2 or target_spin == '13C90':
    nvham.add_spin(np.diag([0,0,0.0888]), nvham.h_13c(), [0, 1])
if len(dims) > 2 or target_spin == '13C13':
    nvham.add_spin(np.diag([0,0, 0.0123]), nvham.h_13c(), [0, 1])

n14_offset = 0.5 * nvham.hf_para_n['14n'] * OC.misc.STz if len(dims) == 5 else 0*qeye(3)
erotframe = get_sub_matrix(nvham.h_e + n14_offset, [1, 2])

def h_mhz(det):
    nrotframe = det * OC.misc.SDz + (get_sub_matrix(nvham.h_13c() + np.array([[0, 0], [0, target_Azz]]), [0, 1]))
    h1 = [tensor(*[erotframe, qeye(2), qeye(2), qeye(2), qeye(2)][:len(dims)]),
          tensor(*[qeye(2), nrotframe, qeye(2), qeye(2), qeye(2)][:len(dims)]),
          tensor(*[qeye(2), qeye(2), nrotframe, qeye(2), qeye(2)][:len(dims)]),
          tensor(*[qeye(2), qeye(2), qeye(2), nrotframe, qeye(2)][:len(dims)]),
          tensor(*[qeye(2), qeye(2), qeye(2), qeye(2), nrotframe][:len(dims)])
          ][:len(dims)]
    h1s = qsum(h1)
    out = nvham.h_nv - h1s
    out = do_zero(out)
    out = Qobj(np.diag(np.diag(out.data.todense())), dims=[dims, dims])
    return out

def H_drift(det):
    return matlab.double(np.array((h_mhz(det)*2*np.pi).data.todense(), dtype=np.complex64).tolist(), is_complex=True)

H_drift_list = [H_drift(det) for det in detunings]
H_ctrl_list  = [[matlab.double(np.array(p[i]*Bc, dtype=np.complex64).tolist(), is_complex=True) for i, Bc in enumerate(L_Bc)] for p in p_scales]

initial_gate = matlab.double(np.eye(D).tolist(), is_complex=True)

pdc_crot0 = dict(dims=dims,
                 rotation_axis={'z': 1},
                 rotated_spin=0,
                 angle=np.pi / 2.,
                 selective_to={target_spin_number: [1]})
pdc_crot1 = dict(dims=dims,
                 rotation_axis={'z': 1},
                 rotated_spin=0,
                 angle=-np.pi / 2.,
                 selective_to={target_spin_number: [0]})
pdc_rotzn = dict(dims=dims,
                 rotation_axis={'z': 1},
                 rotated_spin=target_spin_number,
                 angle=-np.pi / 2.)
pdc_rotyn = dict(dims=dims,
                 rotation_axis={'x': 1},
                 rotated_spin=target_spin_number,
                 angle=-np.pi / 2.)
a0 = get_rot_operator_all_spins(**pdc_crot0)
a1 = get_rot_operator_all_spins(**pdc_crot1)
b = get_rot_operator_all_spins(**pdc_rotyn)
c = get_rot_operator_all_spins(**pdc_rotzn)

#CROT =  matlab.double((b.dag() * a0 * a1 * b * c).data.todense().tolist(), is_complex=True)
CROT =  matlab.double(np.eye(2**len(dims)).tolist(), is_complex=True)

TL = [[T/(n_bins-n_pi), 0.0]]*n_bins if n_bins > 0 else []
TL = np.array(TL)
#for i, val in enumerate(dd_alpha.l_active(n_pi=n_pi, spt=spt)):
for i, val in enumerate(l_active(n_pi=n_pi, spt=spt)):
    if val == 0:
        TL[i][0] = .5*rabi_period
TL = TL.tolist()
TL = matlab.double(TL + [[T_lb, 0]]*(add_slices))

fields[:n_bins, 4:] = 0
if add_slices > 1:
    fields[n_bins:, 4:] = np.diag(np.diag(fields[n_bins:, 4:]))
fields[n_bins:, :4] = 0

#u_array = dd_alpha.u_array_pi(T=T, n_pi=n_pi, spt=spt, target_Azz=target_Azz, rabi_period=rabi_period)
u_array = u_array_pi(T=T, n_pi=n_pi, spt=spt, target_Azz=target_Azz, rabi_period=rabi_period)
u_array = 0*u_array
fields[:n_bins, 2:4] = u_array

fields = matlab.double(np.array(fields, dtype=np.complex64).tolist(), is_complex=True)

###############################################################################
# setup dynamo
###############################################################################
task = 'closed gate'
desc = 'C0ROT1-gate'

reload(OC.dynamo_helpers)
# dp = OC.dynamo_helpers.DynPython('D:\Dropbox\Dokumente\PI3\Code\Matlab', add_slices=add_slices)
dp = DynPython('D:\Dropbox\Dokumente\PI3\Code\Matlab', add_slices=add_slices)
dp.set_dyn(task, initial_gate, CROT, H_drift_list, H_ctrl_list, weight)

c_labels = (['xmw', 'ymw', 'xrf', 'yrf'] + ['z{}'.format(i) for i in range(nz)])
title = 'HIHIHIH'
dp.set_labels(title, dims, c_labels)
dp.seq_init(n_bins, add_slices, TL, control_type, control_par)

###############################################################################
# generate mask
###############################################################################

mask = False*dp.get_mask(optimize_times=False)
#mask[:n_bins, :-1] = 0*dd_alpha.mask_rf(n_pi=n_pi, spt=spt, add_slices=add_slices)
mask[:n_bins, :-1] = 0*mask_rf(n_pi=n_pi, spt=spt, add_slices=add_slices)
if add_slices > 1:
    mask[n_bins:, 4:-1] = np.eye(add_slices)#np.diag(np.diag(mask_rf[n_bins:, 4:]))
else:
    mask[n_bins:, 4:-1] = np.ones(add_slices)  # np.diag(np.diag(mask_rf[n_bins:, 4:]))

###############################################################################
# start optimization
###############################################################################

dp.set_controls(fields)
# dp.open_ui()
dp.search(mask, options=dict(max_walltime=7200., plot_interval=0.5, StepTolerance=1e-13))
# dp.export_mask_list = [dd_alpha.mask_rf(n_pi=n_pi, spt=spt, add_slices=add_slices),
#                        dd_alpha.mask_mw(n_pi=n_pi, spt=spt, add_slices=add_slices)]
dp.export_mask_list = [mask_rf(n_pi=n_pi, spt=spt, add_slices=add_slices),
                       mask_mw(n_pi=n_pi, spt=spt, add_slices=add_slices)]
dp.fields_names_list = ['RF', 'MW']
dp.save(script_path=__file__, script_code=script_code, substr="no_desc_right_now")

Target operation: unitary gate (ignoring global phase) in a closed system.
Hilbert space dimension: 32


64
Controls per slot: 8 +

Error (1): 0.72981
Optimization space dimension: 4

Optimizing algorithm: BFGS (fminunc). Running...

Error (1): 0.72981
Error (1): 0.543857
Error (1): 0.97084
Error (1): 0.587588
Error (1): 0.984403
Error (1): 0.981306
Error (1): 0.0191084
Error (1): 0.0269525
Error (1): 0.00143097
Error (1): 0.00417246
Error (1): 1.07131e-05
Error (1): 1.07002e-05
Error (1): 1.07003e-05
Error (1): 1.07e-05
Error (1): 1.07e-05
Error (1): 1.07e-05

Local minimum found.

Optimization completed because the size of the gradient is less than
the selected value of
Optimal control saved.
qutip_enhanced saved.
Fields RF saved.
Fields MW saved.
add_slices_angles saved.
